In [1]:
!python --version

Python 3.10.9


### Environnement d'exécution

In [2]:
import matplotlib.cm     as cm
import matplotlib.pyplot as plt
import numpy             as np
import os
import scipy.stats       as scs

### Fonctions génériques de traitement d'image

In [3]:
def autocorrelation(img: np.ndarray) -> np.ndarray:
    img_ = np.fft.fft2(img)
    img_ = np.power(np.abs(img_), 2)
    img_ = np.fft.ifft2(img_)
    img_ = np.abs(np.fft.fftshift(img_)/np.nanmax(img_))
    return img_

In [4]:
def cinfinity(img: np.ndarray) -> float:
    cinf = np.power(img.mean(), 2) /np.power(img, 2).mean()
    return cinf

In [5]:
def frequences(img: np.ndarray, shift=False) -> np.ndarray:
    img_ = np.fft.fft2(img)
    if shift: img_ = np.fft.fftshift(img_)
    img_ = np.abs(img_)
    return img_

In [6]:
def spatial(img: np.ndarray, shift=False) -> np.ndarray:
    img_ = np.fft.ifft2(img)
    if shift: img_ = np.fft.fftshift(img_)
    img_ = np.abs(img_)
    return img_

In [7]:
def profile(img: np.ndarray, theta: float) -> tuple:

    while theta > 2*np.pi:
        theta -= 2*np.pi
    
    while theta < 2*np.pi:
        theta += 2*np.pi

    if theta > np.pi:
        theta -= np.pi

    ny, nx = img.shape
    
    j_ = np.linspace(0, nx//2-1, nx//2).astype(np.int32)
    i_ = np.round(j_ *np.tan(theta)).astype(np.int32)

    j_ = j_[np.abs(i_) < ny//2]
    i_ = i_[np.abs(i_) < ny//2]

    radius = np.sqrt( np.power(i_, 2) + np.power(j_, 2) )

    if theta < np.pi/2:
        values = img[ny//2-i_, nx//2+j_]
    else:
        values = img[ny//2-i_, nx//2-j_]

    return (radius, values)

In [8]:
def distance(img: np.ndarray, cinf: float) -> tuple:

    ny, nx = img.shape
    
    angles = np.linspace(0, 179, 180).astype(np.int32)
    radius = np.zeros_like(angles, dtype=np.float64)

    j_ = np.linspace(0, nx//2-1, nx//2).astype(np.int32)

    for k, angle in enumerate(angles):
        
        if angle == 90:
            radius[k] = radius[k-1]
            continue

        i_ = np.round(j_ *np.tan(angle*np.pi/180)).astype(np.int32)
        j_ = j_[np.abs(i_) < ny//2]
        i_ = i_[np.abs(i_) < ny//2]

        radius[k] = np.sqrt( i_[-1]**2 + j_[-1]**2 )
        
        if angle < 90:
            for (i, j) in zip(i_,j_):
                if img[ny//2-i,nx//2+j] > cinf:
                    continue
                else:
                    radius[k] = np.sqrt( i**2 + j**2 )
                    break
        
        else:
            for (i, j) in zip(i_,j_):
                if img[ny//2-i,nx//2-j] >= cinf:
                    continue
                else:
                    radius[k] = np.sqrt( i**2 + j**2 )
                    break

    
    return (angles, radius)

### Fonctions d'affichage génériques

### Implémentation d'une expérience

In [9]:
class Experience():

    def __init__(self, inpath: str, outpath: str):
        self.path = inpath
        self.__read()
        self.__clean()
        self.__filter()
        return
    


    def __read(self):
        self.data = np.genfromtxt( self.path, delimiter=",", dtype=np.float64 )
        self.ny, self.nx = self.data.shape
        return
    

    def __clean(self):
        self.data = np.nan_to_num(self.data, nan=np.nanmean(self.data))
        return
    

    def __filter(self):
        self.img = frequences(self.data, shift=True)
        self.img[:,self.nx//2] = np.power(self.img[:,self.nx//2], 0.5)
        self.img = spatial(self.img, shift=False)
        return



    def analyse(self, folder=""):
        
        self.describe(folder)
        

        nrows = 1
        ncols = 4
        fig = plt.figure(figsize=(nrows*40, ncols*(8+0.5)))

        axe = fig.add_subplot(nrows, ncols, 1)
        axe.imshow(self.data, origin="lower")
        axe.set_title("$E_{yy}$")
        axe.set_xlabel("X [px]")
        axe.set_ylabel("Y [px]")


        freqs = frequences(self.data)

        axe = fig.add_subplot(nrows, ncols, 2)
        axe.imshow(np.log(freqs), origin="lower")
        axe.set_title("$\log(\mathcal{F}(E_{yy}))$")
        axe.set_xlabel("X [px]")
        axe.set_ylabel("Y [px]")


        axe = fig.add_subplot(nrows, ncols, 3)
        axe.imshow(np.log(self.img), origin="lower")
        axe.set_title("$\log(\mathcal{T}(E_{yy}))$")
        axe.set_xlabel("X [px]")
        axe.set_ylabel("Y [px]")


        autocor = autocorrelation(self.img)
        cinf    = cinfinity(self.data)

        axe = fig.add_subplot(nrows, ncols, 4)
        # axe.imshow(np.log(autocor/cinf), origin='lower')
        # axe.contour(np.log(autocor/cinf), [0.0, (autocor/cinf).max()], cmap=cm.jet)
        axe.imshow(autocor, origin='lower')
        axe.contour(autocor, [cinf, autocor.max()], cmap=cm.jet)
        # axe.set_title("$\log\left(\\frac{\mathcal{A}(E_{yy})}{C_\infty}\\right)$")
        axe.set_title("$\mathcal{A}(E_{yy})$")
        axe.set_xlabel("X [px]")
        axe.set_ylabel("Y [px]")

        plt.suptitle(f"{os.path.basename(self.path)}", fontsize=40)

        if folder == "":
            plt.show()
        else:
            plt.savefig(f"{folder}/{os.path.basename(self.path)}_autocorrelation.jpg")
        
        plt.close()


        fig = plt.figure()

        angles, radius = distance(autocor, cinf)

        axe = fig.add_subplot(1,1,1)
        axe.plot(angles, radius)
        axe.set_title("Distance au contour en fonction de la direction")
        axe.set_xlabel("Angle [°]")
        axe.set_xlim(left=0, right=180)
        axe.set_ylabel("Radius to $C_\infty$ [px]")
        axe.set_ylim(bottom=0, top=radius.max())

        if folder == "":
            plt.show()
        else:
            plt.savefig(f"{folder}/{os.path.basename(self.path)}_radius_vs_angle.jpg")
        
        plt.close()

        return


    def describe(self, path=""):
        if path == "":
            print(f"Data description :")
            print(f"------------------")
            print(f"dimension : {self.data.ndim}")
            print(f"shape     : {self.data.shape}")
            print(f"NaNs      : {np.any(np.isnan(self.data))}")
            print(f"mean      : {np.nanmean(self.data)}")
            print(f"std       : {np.nanstd(self.data)}")
            print(f"min       : {np.nanmin(self.data)}")
            print(f"max       : {np.nanmax(self.data)}")
            print(f"kurtosis  : {scs.kurtosis(self.data.flatten(), fisher=True, nan_policy='omit')}")
            print(f"skewness  : {scs.skew(self.data.flatten(), nan_policy='omit')}")
            print(f"C_inf     : {cinfinity(self.data)}")
            print()
            print(f"Image description :")
            print(f"-------------------")
            print(f"dimension : {self.img.ndim}")
            print(f"shape     : {self.img.shape}")
            print(f"NaNs      : {np.any(np.isnan(self.img))}")
            print(f"mean      : {np.nanmean(self.img)}")
            print(f"std       : {np.nanstd(self.img)}")
            print(f"min       : {np.nanmin(self.img)}")
            print(f"max       : {np.nanmax(self.img)}")
            print(f"kurtosis  : {scs.kurtosis(self.img.flatten(), fisher=True, nan_policy='omit')}")
            print(f"skewness  : {scs.skew(self.img.flatten(), nan_policy='omit')}")
            print(f"C_inf     : {cinfinity(self.img)}")

        else:
            with open(os.path.join(path, f"{os.path.basename(self.path)}.txt"), "w") as file:
                file.write(f"Data description :\n")
                file.write(f"------------------\n")
                file.write(f"dimension : {self.data.ndim}\n")
                file.write(f"shape     : {self.data.shape}\n")
                file.write(f"NaNs      : {np.any(np.isnan(self.data))}\n")
                file.write(f"mean      : {np.nanmean(self.data)}\n")
                file.write(f"std       : {np.nanstd(self.data)}\n")
                file.write(f"min       : {np.nanmin(self.data)}\n")
                file.write(f"max       : {np.nanmax(self.data)}\n")
                file.write(f"kurtosis  : {scs.kurtosis(self.data.flatten(), fisher=True, nan_policy='omit')}\n")
                file.write(f"skewness  : {scs.skew(self.data.flatten(), nan_policy='omit')}\n")
                file.write(f"C_inf     : {cinfinity(self.data)}\n")
                file.write("\n")
                file.write(f"Image description :\n")
                file.write(f"-------------------\n")
                file.write(f"dimension : {self.img.ndim}\n")
                file.write(f"shape     : {self.img.shape}\n")
                file.write(f"NaNs      : {np.any(np.isnan(self.img))}\n")
                file.write(f"mean      : {np.nanmean(self.img)}\n")
                file.write(f"std       : {np.nanstd(self.img)}\n")
                file.write(f"min       : {np.nanmin(self.img)}\n")
                file.write(f"max       : {np.nanmax(self.img)}\n")
                file.write(f"kurtosis  : {scs.kurtosis(self.img.flatten(), fisher=True, nan_policy='omit')}\n")
                file.write(f"skewness  : {scs.skew(self.img.flatten(), nan_policy='omit')}\n")
                file.write(f"C_inf     : {cinfinity(self.img)}\n")
        return


    def plot_data(self, path=""):
        fig = plt.figure()
        
        axe = fig.add_subplot(1,1,1)
        axe.axis("equal")
        axe.imshow(self.data, origin="lower")
        axe.set_title(f"{os.path.basename(self.path)} - données brutes")
        axe.set_xlabel("X [px]")
        axe.set_ylabel("Y [px]")
        
        if path == "":
            plt.show()
        else:
            plt.savefig(f"{path}/donnees.jpg")

        plt.close()
        return
    

    def plot_img(self, path=""):
        fig = plt.figure()
        
        axe = fig.add_subplot(1,1,1)
        axe.axis("equal")
        axe.imshow(np.log(self.img), origin="lower")
        axe.set_title(f"{os.path.basename(self.path)} - données pré-traitées ($\log$)")
        axe.set_xlabel("X [px]")
        axe.set_ylabel("Y [px]")
        
        if path == "":
            plt.show()
        else:
            plt.savefig(f"{path}/image.jpg")

        plt.close()
        return

### Validation de la classe `Experience` et chemin de données du post-traitement

In [10]:
infile = "C:/Users/julie/Documents/experiences/TD8_Eyy_point419_1.csv"
path, name = os.path.split(infile)

experience = Experience(infile, "")

experience.analyse(folder=path)